In [1]:
import os
import sys
sys.path.append('..')
LIB_PATH = os.getcwd() + "/art_library"
sys.path.append(LIB_PATH)
print(*sys.path, sep='\n')

/home/lukec/workspace/baard_exp2/experiments
/home/lukec/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles
/home/lukec/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles/lib/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/workspace/baard_exp2/venv/lib/python3.6/site-packages
/home/lukec/workspace/baard_exp2/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython
..
/home/lukec/workspace/baard_exp2/experiments/art_library


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json

import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd

In [4]:
from models.cifar10 import  Resnet, Vgg
from models.torch_util import predict_numpy
from defences.util import acc_on_adv
from defences.baard import ApplicabilityStage, ReliabilityStage, DecidabilityStage, BAARDOperator

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


In [6]:
resnet = Resnet(use_prob=True).to(device)
vgg = Vgg(use_prob=True).to(device)

In [7]:
f_resnet = os.path.join('..', 'results', 'cifar10_resnet_200.pt')
f_vgg = os.path.join('..', 'results', 'cifar10_vgg_200.pt')

resnet.load_state_dict(torch.load(f_resnet, map_location=device))
vgg.load_state_dict(torch.load(f_vgg, map_location=device))

<All keys matched successfully>

In [8]:
# Create BAARD
f_json_param = os.path.join('..', 'params', 'baard_cifar10_3.json')
with open(f_json_param) as j:
    baard_param = json.load(j)
print('Param:', baard_param)

n_classes = 10

Param: {'k_re': 10, 'k_de': 100, 'q1': 1.0, 'q2': 0.98, 'q3': 0.98, 'sequence': [True, True, True]}


In [9]:

stages_resnet = []
stages_resnet.append(ApplicabilityStage(n_classes=n_classes, quantile=baard_param['q1'], verbose=0))
stages_resnet.append(ReliabilityStage(n_classes=n_classes, k=baard_param['k_re'], quantile=baard_param['q2'], verbose=0))
stages_resnet.append(DecidabilityStage(n_classes=n_classes, k=baard_param['k_de'], quantile=baard_param['q3'], verbose=0))
detector_resnet = BAARDOperator(stages=stages_resnet)

stages_vgg = []
stages_vgg.append(ApplicabilityStage(n_classes=n_classes, quantile=baard_param['q1'], verbose=0))
stages_vgg.append(ReliabilityStage(n_classes=n_classes, k=baard_param['k_re'], quantile=baard_param['q2'], verbose=0))
stages_vgg.append(DecidabilityStage(n_classes=n_classes, k=baard_param['k_de'], quantile=baard_param['q3'], verbose=0))
detector_vgg = BAARDOperator(stages=stages_vgg)

In [10]:
# Load pretrained parameters
f_pre_data = os.path.join('..', 'result_0', 'cifar10_resnet_baard_s1_train_data.pt')
obj = torch.load(f_pre_data)
X_train_s1 = obj['X_s1']
X_train = obj['X']
y_train = obj['y']
print(X_train_s1.shape, X_train.shape, y_train.shape)

(48179, 3, 32, 32) (48179, 3, 32, 32) (48179,)


In [11]:
f_thresh_resnet = os.path.join('..', 'result_0', 'cifar10_resnet_baard_threshold.pt')
detector_resnet.fit(X=X_train, y=y_train, X_s1=X_train_s1)
detector_resnet.load(f_thresh_resnet)

f_thresh_vgg = os.path.join('..', 'result_0', 'cifar10_vgg_baard_threshold.pt')
detector_vgg.fit(X=X_train, y=y_train, X_s1=X_train_s1)
detector_vgg.load(f_thresh_vgg)

Load from: ../result_0/cifar10_resnet_baard_threshold.pt
Load from: ../result_0/cifar10_vgg_baard_threshold.pt


In [12]:
def get_data(model_name, att, e, d='adv'):
    f = 'cifar10_{}_{}_{}_{}.npy'.format(model_name, att, e, d)
    p = os.path.join('..', 'results', f)
    return np.load(p)

In [13]:
get_data('resnet', 'apgd', '0.031').shape

(2000, 3, 32, 32)

In [14]:
def get_transfer(att, eps):
    acc_att_vgg = []
    acc_det_vgg = []
    acc_att_res = []
    acc_det_res = []

    for e in eps:
        adv_res = get_data('resnet', att, e)[:1000]
        y_res = get_data('resnet', att, e, d='y')[:1000]
        pred_res_from_vgg = predict_numpy(vgg, adv_res, device)[:1000]
        acc_vgg = np.mean(pred_res_from_vgg == y_res)
        print('[{} eps={:.3f}] {:6s} acc on adv. examples transfered from {:6s}: {}'.format(att, e, 'vgg', 'resnet', acc_vgg))
        labelled_as_adv_vgg = detector_vgg.detect(adv_res, pred_res_from_vgg)
        acc_det = acc_on_adv(pred_res_from_vgg, y_res, labelled_as_adv_vgg)
        print('[{} eps={:.3f}] {:6s} acc_on_adv: {}'.format(att, e, 'vgg', acc_det))
        acc_att_vgg.append(acc_vgg)
        acc_det_vgg.append(acc_det)

        adv_vgg = get_data('vgg', att, e)[:1000]
        y_vgg = get_data('vgg', att, e, d='y')[:1000]
        pred_vgg_from_res = predict_numpy(resnet, adv_vgg, device)[:1000]
        acc_res = np.mean(pred_vgg_from_res == y_vgg)
        print('[{} eps={:.3f}] {:6s} acc on adv. examples transfered from {:6s}: {}'.format(att, e, 'resnet', 'vgg', acc_res))
        labelled_as_adv_res = detector_resnet.detect(adv_vgg, pred_vgg_from_res)
        acc_det = acc_on_adv(pred_vgg_from_res, y_vgg, labelled_as_adv_res)
        print('[{} eps={:.3f}] {:6s} acc_on_adv: {}'.format(att, e, 'resnet', acc_det))
        acc_att_res.append(acc_res)
        acc_det_res.append(acc_det)
    return acc_att_vgg, acc_det_vgg, acc_att_res, acc_det_res

In [19]:
def get_df(att, eps, att_vgg, det_vgg, att_res, det_res):
    data = {
        'Attack': np.repeat(att, len(eps)), 
        'Adv_param': eps, 
        'VGG Acc. from ResNet Att.': att_vgg, 
        'VGG BAARD Acc.': det_vgg, 
        'ResNet Acc. from VGG Att.': att_res, 
        'ResNet BAARD Acc.': det_res
    }
    return pd.DataFrame(data=data)
    

In [20]:
df = get_df('psudo_att', np.arange(5), np.random.rand(5), np.random.rand(5), np.random.rand(5), np.random.rand(5))
df

,Attack,Adv_param,VGG Acc. from ResNet Att.,VGG BAARD Acc.,ResNet Acc. from VGG Att.,ResNet BAARD Acc.
0,psudo_att,0,0.192018,0.460727,0.930759,0.608808
1,psudo_att,1,0.701342,0.701816,0.087311,0.817822
2,psudo_att,2,0.974281,0.909371,0.873683,0.386539
3,psudo_att,3,0.258772,0.072712,0.091747,0.478941
4,psudo_att,4,0.022978,0.484209,0.528354,0.134151


In [21]:
eps = [0.031, 0.3, 0.6, 1.0, 1.5]
att = 'apgd'
apgd_acc_att_vgg, apgd_acc_det_vgg, apgd_acc_att_res, apgd_acc_det_res = get_transfer(att, eps)
apgd_df = get_df(att, eps, apgd_acc_att_vgg, apgd_acc_det_vgg, apgd_acc_att_res, apgd_acc_det_res)
print(apgd_df)
apgd_df.to_csv('trans_{}.csv'.format(att), index=False)

[apgd eps=0.031] vgg    acc on adv. examples transfered from resnet: 0.855
[apgd eps=0.031] vgg    acc_on_adv: 0.867
[apgd eps=0.031] resnet acc on adv. examples transfered from vgg   : 0.302
[apgd eps=0.031] resnet acc_on_adv: 0.345
[apgd eps=0.300] vgg    acc on adv. examples transfered from resnet: 0.075
[apgd eps=0.300] vgg    acc_on_adv: 0.362
[apgd eps=0.300] resnet acc on adv. examples transfered from vgg   : 0.0
[apgd eps=0.300] resnet acc_on_adv: 0.609
[apgd eps=0.600] vgg    acc on adv. examples transfered from resnet: 0.026
[apgd eps=0.600] vgg    acc_on_adv: 0.935
[apgd eps=0.600] resnet acc on adv. examples transfered from vgg   : 0.016
[apgd eps=0.600] resnet acc_on_adv: 1.0
[apgd eps=1.000] vgg    acc on adv. examples transfered from resnet: 0.03
[apgd eps=1.000] vgg    acc_on_adv: 1.0
[apgd eps=1.000] resnet acc on adv. examples transfered from vgg   : 0.051
[apgd eps=1.000] resnet acc_on_adv: 1.0
[apgd eps=1.500] vgg    acc on adv. examples transfered from resnet: 0.02

In [25]:
eps = [1.5, 2.0, 3.0, 5.0]
att = 'apgd2'
apgd2_acc_att_vgg, apgd2_acc_det_vgg, apgd2_acc_att_res, apgd2_acc_det_res = get_transfer(att, eps)
apgd2_df = get_df(att, eps, apgd2_acc_att_vgg, apgd2_acc_det_vgg, apgd2_acc_att_res, apgd2_acc_det_res)
print(apgd2_df)
apgd2_df.to_csv('trans_{}.csv'.format(att))

[apgd2 eps=1.500] vgg    acc on adv. examples transfered from resnet: 0.77
[apgd2 eps=1.500] vgg    acc_on_adv: 0.789
[apgd2 eps=1.500] resnet acc on adv. examples transfered from vgg   : 0.213
[apgd2 eps=1.500] resnet acc_on_adv: 0.27
[apgd2 eps=2.000] vgg    acc on adv. examples transfered from resnet: 0.676
[apgd2 eps=2.000] vgg    acc_on_adv: 0.706
[apgd2 eps=2.000] resnet acc on adv. examples transfered from vgg   : 0.106
[apgd2 eps=2.000] resnet acc_on_adv: 0.172
[apgd2 eps=3.000] vgg    acc on adv. examples transfered from resnet: 0.459
[apgd2 eps=3.000] vgg    acc_on_adv: 0.539
[apgd2 eps=3.000] resnet acc on adv. examples transfered from vgg   : 0.035
[apgd2 eps=3.000] resnet acc_on_adv: 0.129
[apgd2 eps=5.000] vgg    acc on adv. examples transfered from resnet: 0.225
[apgd2 eps=5.000] vgg    acc_on_adv: 0.392
[apgd2 eps=5.000] resnet acc on adv. examples transfered from vgg   : 0.007
[apgd2 eps=5.000] resnet acc_on_adv: 0.168
  Attack  Adv_param  VGG Acc. from ResNet Att.  VG

In [28]:
eps = [0.0, 5.0, 10.0]
att = 'cw2'
cw2_acc_att_vgg, cw2_acc_det_vgg, cw2_acc_att_res, cw2_acc_det_res = get_transfer(att, eps)
cw2_df = get_df(att, eps, cw2_acc_att_vgg, cw2_acc_det_vgg, cw2_acc_att_res, cw2_acc_det_res)
print(cw2_df)
cw2_df.to_csv('trans_{}.csv'.format(att))

[cw2 eps=0.000] vgg    acc on adv. examples transfered from resnet: 0.947
[cw2 eps=0.000] vgg    acc_on_adv: 0.95
[cw2 eps=0.000] resnet acc on adv. examples transfered from vgg   : 0.893
[cw2 eps=0.000] resnet acc_on_adv: 0.899
[cw2 eps=5.000] vgg    acc on adv. examples transfered from resnet: 0.937
[cw2 eps=5.000] vgg    acc_on_adv: 0.94
[cw2 eps=5.000] resnet acc on adv. examples transfered from vgg   : 0.848
[cw2 eps=5.000] resnet acc_on_adv: 0.857
[cw2 eps=10.000] vgg    acc on adv. examples transfered from resnet: 0.906
[cw2 eps=10.000] vgg    acc_on_adv: 0.914
[cw2 eps=10.000] resnet acc on adv. examples transfered from vgg   : 0.798
[cw2 eps=10.000] resnet acc_on_adv: 0.812
  Attack  Adv_param  VGG Acc. from ResNet Att.  VGG BAARD Acc.  \
0    cw2        0.0                      0.947           0.950   
1    cw2        5.0                      0.937           0.940   
2    cw2       10.0                      0.906           0.914   

   ResNet Acc. from VGG Att.  ResNet BAARD 

In [29]:
eps = [0.031, 0.3, 0.6, 1.0, 1.5]
att = 'fgsm'
fgsm_acc_att_vgg, fgsm_acc_det_vgg, fgsm_acc_att_res, fgsm_acc_det_res = get_transfer(att, eps)

[fgsm eps=0.031] vgg    acc on adv. examples transfered from resnet: 0.814
[fgsm eps=0.031] vgg    acc_on_adv: 0.831
[fgsm eps=0.031] resnet acc on adv. examples transfered from vgg   : 0.419
[fgsm eps=0.031] resnet acc_on_adv: 0.451
[fgsm eps=0.300] vgg    acc on adv. examples transfered from resnet: 0.152
[fgsm eps=0.300] vgg    acc_on_adv: 0.986
[fgsm eps=0.300] resnet acc on adv. examples transfered from vgg   : 0.121
[fgsm eps=0.300] resnet acc_on_adv: 0.972
[fgsm eps=0.600] vgg    acc on adv. examples transfered from resnet: 0.128
[fgsm eps=0.600] vgg    acc_on_adv: 1.0
[fgsm eps=0.600] resnet acc on adv. examples transfered from vgg   : 0.117
[fgsm eps=0.600] resnet acc_on_adv: 1.0
[fgsm eps=1.000] vgg    acc on adv. examples transfered from resnet: 0.111
[fgsm eps=1.000] vgg    acc_on_adv: 1.0
[fgsm eps=1.000] resnet acc on adv. examples transfered from vgg   : 0.121
[fgsm eps=1.000] resnet acc_on_adv: 1.0
[fgsm eps=1.500] vgg    acc on adv. examples transfered from resnet: 0.1

ValueError: arrays must all be same length

In [35]:
fgsm_df = get_df(att, eps, fgsm_acc_att_vgg, fgsm_acc_det_vgg, fgsm_acc_att_res, fgsm_acc_det_res)
print(fgsm_df)
fgsm_df.to_csv('trans_{}.csv'.format(att))

  Attack  Adv_param  VGG Acc. from ResNet Att.  VGG BAARD Acc.  \
0   fgsm      0.031                      0.814           0.831   
1   fgsm      0.300                      0.152           0.986   
2   fgsm      0.600                      0.128           1.000   
3   fgsm      1.000                      0.111           1.000   
4   fgsm      1.500                      0.111           1.000   

   ResNet Acc. from VGG Att.  ResNet BAARD Acc.  
0                      0.419              0.451  
1                      0.121              0.972  
2                      0.117              1.000  
3                      0.121              1.000  
4                      0.121              1.000  
